<a href="https://colab.research.google.com/github/rajviishah/CMPE-297-Advanced-Deep-Learning/blob/main/Assignment%205/CMPE_297_Assignment_5_part_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **a) Do an end2end well annotated and commented demo of continual learning using avalanche library in colab**

## **SimpleMLP modeling on PermutedMNIST dataset**

Avalanche and its dependencies can be installed using the following command. Avalanche runs on either CPU or CUDA GPU based on availability. Since Avalanche is built on top of PyTorch, a PyTorch environment has to be created.

In [2]:
# Install Avalanche and its dependencies
!pip install git+https://github.com/ContinualAI/avalanche.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-lnhamgv0
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-lnhamgv0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


Create the environment by importing necessary libraries and modules.

In [4]:
import torch
# use CrossEntropyLoss
from torch.nn import CrossEntropyLoss
# use stochastic GD optimizer
from torch.optim import SGD
# import the PermutedMNIST dataset
from avalanche.benchmarks.classic import PermutedMNIST
# import the SimpleMLP dataset
from avalanche.models import SimpleMLP
# we will use Naive training strategy
from avalanche.training.supervised import Naive

Configure the device settings by checking for availability of CUDA GPU. Instantiate a simple multi-layer perceptron model.

In [5]:
# Configure the device settings
# check for CUDA GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model instantiation
model = SimpleMLP(num_classes = 10)

Prepare the benchmark dataset. Split the dataset into train and test sets.

In [6]:
# Benchmark instantiation
perm_mnist = PermutedMNIST(n_experiences=3)
# split stream into train and test set
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



Instantiate optimizer and loss functions.

In [7]:
# define optimizer and loss function for training
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

Devise a training strategy. Here, the Naive strategy is implemented.

In [8]:
# Instantiate Continual learning strategy
strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=2,
    eval_mb_size=32, device=device)

Perform training, evaluate the model and log results.

In [9]:
results = []
# train and test epochs
for train_task in train_stream:
    # training
    strategy.train(train_task, num_workers=4)
    # evaluate the model and store the results
    results.append(strategy.eval(test_stream))

-- >> Start of training phase << --
0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 1875/1875 [01:06<00:00, 28.39it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4932
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8594
100%|██████████| 1875/1875 [01:04<00:00, 28.91it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2583
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9249
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 313/313 [00:05<00:00, 54.60it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1845
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9464
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 313/313 [00:05<00:00, 54.68it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 2.7439
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0854
-- S

In [10]:
results

[{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9248833333333333,
  'Loss_Epoch/train_phase/train_stream/Task000': 0.2583487089931965,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9464,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.18446238237023355,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.0854,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp001': 2.7439272869110107,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002': 0.1156,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp002': 2.577794418334961,
  'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.3824666666666667,
  'Loss_Stream/eval_phase/test_stream/Task000': 1.8353946958720684},
 {'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.93495,
  'Loss_Epoch/train_phase/train_stream/Task000': 0.22539884010851383,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9391,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.2016583152666688,
  'Top1_Acc_Exp/eval_phase/test_stream/Ta

**Avalanche provides the strategy of clean and simple code that makes it easy to use.**

Reference: https://analyticsindiamag.com/avalanche-a-python-library-for-continual-learning/